# Data Science in Psychology & Neuroscience (DSPN): 

## Lecture 15. Data Visualization (part 2)

### Date: October 19, 2023

### To-Dos From Last Class:

* Install R/RStudio (or IRKernel/Jupyter)
    
### Today:

1. Plot some data
    1. ggplot template
    2. Layers
    3. Grouping the data (Fill & Color)
    4. Other formatting stuff (Themes)
    5. Spotting / labelling outliers

### Homework

* Brainstorm Final Assignment ideas :)

## Reminder: Imitation inhibition task dataset

<img src="img/ait_task.png" width=500>

* 20 trials per condition (100 total responses for each participant)
    * Average across cued response finger
* Condition mapping:
    1. Baseline 
    2. Effector congruent 
    3. Effector incongruent
    4. Movement congruent
    5. Movement incongruent

# Part 1A: Reading in the dataset

* Load the tidyverse package
* Assign filepath
* Read in the data

In [13]:
library(tidyverse)

# read in the data
d_data <- '/Users/jeremyhogeveen/Dropbox/Fall_2023/teaching/PSY450_650/DSPN_Fall2023_Git/misc/imitation_inhibition_paradigm/data/second'
f_trialwise <- paste(d_data,'ait_trialwise.csv',sep='/')

# read in the dataset
df_trialwise <- read_csv(f_trialwise)

Rows: 4745 Columns: 28
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (11): Group, pid, Name.1, Name.2, Name.3, Response, Key, Released, Respo...
dbl (15): Time, Finger, Congruence, Repeated, double_response, questionnaire...
lgl  (2): (Trial Variable), Correct

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# 1B. Compute summary RT and error rate (ER) data by participant/condition

## <span style="color:blue">NOTE: I will do the wrangling for you in R for Assignment 4 and 5 starters!</span>

* Normalize conditional RTs by "baseline"
* Recode "Congruence" as a 2 x 2 design
    * Stimulus type: Movement or Effector
    * Stimulus/Response Compatibility: Congruent or Incongruent
* Recode "Error Code" as an integer
    * Useful for calculateing ER
* Compute summary statistics

In [21]:
# Normalizing effector / movement conditions by baseline

# pull the baseline, compute base RT for each participant
df_baseline <- df_trialwise %>%
    filter(Congruence==0) %>%
    group_by(pid) %>%
    summarise(RT_base=mean(Time))

# filter to just other conditions, merge baseline RTs back in, use to compute normalized RT
df_wconditions <- df_trialwise %>%
    filter(Congruence != 0) %>%
    merge(df_baseline,by = "pid") %>%
    group_by(pid) %>%
    mutate(RT_sd = sd(Time),
        RT_norm = (Time - RT_base) / sd(Time)) #%>%
#     select(RT_sd,Time,RT_base,RT_norm)
# head(df_conditions,5)

# separating Congruence factor into 2 variables for easy ggplot coding
df_wconditions <- df_wconditions %>%
    mutate(str_condi = ifelse(Congruence==1,'eff_con',
                              ifelse(Congruence==2,'eff_inc',
                                     ifelse(Congruence==3,'move_con',
                                            ifelse(Congruence==4,'move_inc','')))),
           stim_type = ifelse(grepl('^eff',str_condi), "Effector", "Movement"),
           sr_congruence = ifelse(grepl('con$', str_condi), "Congruent", "Incongruent"),
           error = ifelse(Code=="C",0,1))

In [22]:
df_wconditions %>% 
    group_by(stim_type,sr_congruence) %>% 
    summarise(RT_mean = mean(RT_norm),
              N = n(),
              ER_mean = sum(error)/N) 

`summarise()` has grouped output by 'stim_type'. You can override using the
`.groups` argument.


stim_type,sr_congruence,RT_mean,N,ER_mean
<chr>,<chr>,<dbl>,<int>,<dbl>
Effector,Congruent,-0.1857117,944,0.01165254
Effector,Incongruent,0.1729082,946,0.04228330
Movement,Congruent,-0.4572500,946,0.02325581
Movement,Incongruent,0.1143616,957,0.06374086


In [1]:

# getting a data frame summarized by all condis
df_sum <- df_wconditions %>% 
    group_by(pid,str_condi) %>% 
    summarise(N = n(),
              RT_mean = mean(RT_norm),
              ER_mean = sum(error) / N)

ERROR: Error in df_wconditions %>% group_by(pid, str_condi) %>% summarise(N = n(), : could not find function "%>%"


# 2A. ggplot template

* Note: ggplot reference for different geom_functions etc.
    * https://ggplot2.tidyverse.org/reference/

In [ ]:
# ggplot(data = <DATA>) + 
#   <GEOM_FUNCTION>(mapping = aes(<MAPPINGS>)) 

# ggplot(data = <DATA>, aes(<MAPPINGS>)) + 
#   <GEOM_FUNCTION>() 

# ggplot initiates a blank plot object
# aes = aesthetic mappings
# geom_functions = geometric objects

# 2B. Layers

* Can layer as many different geometric objects (i.e., geom_ thingys) as you like!

# 2C. Grouping the data (e.g. for visualizing interactions between factors)

* Plotting stim type BY sr_comp interaction
* Noting the difference between _fill_ and _colour_
    * Fill --> specifying colour to fill a geom
    * Colour --> specifying colour to outline a geom

# 2D. Formatting the plots (Themes)

* Plot outline / background
* Axis labels

# 2E. Spotting + Labeling Outliers

* Plotting speed-accuracy tradeoff scatter
* Labelling points by participant ID to figure out who I should look into further

# 2F. Making a Mean +/- SEM plot __after__ you know your data 

* compute 2x2 RT summary _across_ participants
* plot mean +/- SEM